In [1]:
import torch
torch.backends.cudnn.benchmark=True

import os
from datetime import datetime
import time

from loss import *
from dataset import *
from model import *
from model_resnetyolo import *
from viz import *
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import DataLoader
import torch.optim as optim
from tqdm import tqdm
import matplotlib.pyplot as plt

## some parameters

In [2]:
C=20
B=2
S=7
batch_size = 4
learning_rate = 1e-4
epochs = 120

# need more augmentation!

In [3]:
# https://albumentations.ai/docs/api_reference/augmentations/transforms/
# https://albumentations.ai/docs/api_reference/augmentations/geometric/transforms/
# https://albumentations.ai/docs/getting_started/bounding_boxes_augmentation/
transform_aug = A.Compose([
    A.Resize(448, 448),
    A.HorizontalFlip(),
    A.ColorJitter(brightness=0.2, contrast=0., saturation=0.2, hue=0.15),
    A.Affine(scale=(1.0,1.2),translate_percent=(-0.2,0.2)),
    A.Normalize(), # mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)
    ToTensorV2()
], bbox_params=A.BboxParams(format='albumentations', min_visibility=0.3))

transform = A.Compose([
    A.Resize(448, 448),
    A.Normalize(), # mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)
    ToTensorV2()
], bbox_params=A.BboxParams(format='albumentations')) # format=albumentations is normalized pascal_voc.


train_dataset=VOCDataset(root='data', image_set='train', S=S,C=C,transform=transform_aug)
test_dataset=VOCDataset(root='data', image_set='val',  S=S,C=C,transform=transform)

train_loader=DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, num_workers=8)
test_loader=DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False, num_workers=8)


Using downloaded and verified file: data/VOCtrainval_11-May-2012.tar
Extracting data/VOCtrainval_11-May-2012.tar to data
Using downloaded and verified file: data/VOCtrainval_06-Nov-2007.tar
Extracting data/VOCtrainval_06-Nov-2007.tar to data


In [4]:
# net = Yolov1(split_size=S, num_boxes=B, num_classes=C).cuda()
net = YOLOv1ResNet(S=S, B=B, C=C).cuda()
optimizer = optim.Adam(net.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
loss_fn = YoloLoss(S=S, B=B, C=C)

In [5]:
# net.load_state_dict(torch.load('backup/checkpoint_149.npy'))

## define functions for training and validation

In [6]:
def train():
    net.train()
    train_loss = 0
    loop = tqdm(train_loader)
    for i, (img, (boxes, labels, Iobj), index) in enumerate(loop):
        img, boxes, labels, Iobj = img.cuda(), boxes.cuda(), labels.cuda(), Iobj.cuda()
        out = net(img)
        loss = loss_fn(out, (boxes, labels, Iobj))
        train_loss+=loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    loss_history['train'].append(train_loss / (i + 1))

    
    
@torch.no_grad()
def val_test(dataloader):
    net.eval()
    total_loss = 0
    loop = tqdm(dataloader)
    for i, (img, (boxes, labels, Iobj), index) in enumerate(loop):
        img, boxes, labels, Iobj = img.cuda(), boxes.cuda(), labels.cuda(), Iobj.cuda()
        out = net(img)
        loss = loss_fn(out, (boxes, labels, Iobj))
        total_loss += loss.item()
    
    avg_loss = total_loss / (i + 1)
    return avg_loss
    

    
def save():
    global lowest_loss
    # save checkpoint
    
    if epoch>50 and loss_history['val'][-1]<10 and loss_history['val'][-1] < lowest_loss:
        torch.save(net.state_dict(), f'models/checkpoint_{str(epoch).zfill(3)}.npy')
        lowest_loss = loss_history['val'][-1]

    # save loss, error history and log
    torch.save(loss_history, 'logs/loss_history.npy')
    with open('logs/log.txt', 'a') as f:
        f.write('epoch{} finished at {}. val_loss: {:.4f}\n'.
                format(str(epoch).zfill(3),
                       datetime.now().strftime("%m/%d/%Y, %H:%M:%S"),
                       loss_history['val'][-1]))
        

## start training

In [ ]:
loss_history = {'train': [], 'val': []}

if os.path.exists('logs/log.txt'):
    raise OSError('Previous training logs already exist.')

lowest_loss = 100
warmup=0
for epoch in range(epochs):
    if epoch < warmup:
        optimizer.param_groups[0]['lr'] = learning_rate / 10
    elif epoch == warmup:
        optimizer.param_groups[0]['lr'] = learning_rate
    
    # train
    train()
    
    # validation
    test_loss = val_test(test_loader)
    loss_history['val'].append(test_loss)
    print(' - val_loss: {:.4f}\n'.format(loss_history['val'][-1]))
    
    # save
    save()
    
    # step scheduler
    scheduler.step()

 41%|████████████████                       | 1191/2885 [03:43<05:18,  5.32it/s]

In [ ]:
loss_history = torch.load('logs/loss_history.npy')
plt.plot(loss_history['train'], 'r', label='train')
plt.plot(loss_history['val'], 'b', label='val')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.title('loss history')
plt.legend()
plt.ylim([1,10])
plt.show()

In [ ]:
%%javascript
IPython.notebook.save_notebook()

In [ ]:
# import subprocess
# import time
# time.sleep(3)
# subprocess.Popen(('shutdown', 'now'), stdout=subprocess.PIPE)